In [36]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [48]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
toronto_data = BeautifulSoup(source, 'lxml')

In [49]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [94]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n',).replace('\n','')
            
    toronto = toronto.append({'Postalcode': postcode ,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                               
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(','.join).reset_index()
df = df.replace(r'\n',' ', regex=True)

df2= pd.DataFrame(df)

df2 = df2.dropna()
empty = 'Not assigned'
df2 = df2[(df2.Postalcode != empty ) & (df2.Borough != empty) & (df2.Neighborhood != empty)]

def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df2.groupby(['Postalcode', 'Borough'])
df3 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [95]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [96]:
df_lon_lat.columns=['Postalcode','Latitude','Longitude']

In [97]:
Toronto_df = pd.merge ( df3, df_lon_lat, on='Postalcode')

In [98]:
Toronto_df

,Postalcode,Borough,Neighborhood,Latitude,Longitude
